In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.2 MB/s eta 0:00:00


In [2]:
!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 502.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 46.1 MB/s eta 0:00:00
  Created wheel

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv
from sklearn.preprocessing import StandardScaler

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir FreeSolv_dataset

In [6]:
!unzip /content/drive/MyDrive/freesolv.zip -d FreeSolv_dataset

Archive:  /content/drive/MyDrive/freesolv.zip
  inflating: FreeSolv_dataset/scaffold_0_smiles_train.pickle  
  inflating: FreeSolv_dataset/scaffold_0_test.bin  
  inflating: FreeSolv_dataset/scaffold_0_val.bin  
  inflating: FreeSolv_dataset/scaffold_0_smiles_val.pickle  
  inflating: FreeSolv_dataset/scaffold_0_smiles_test.pickle  
  inflating: FreeSolv_dataset/scaffold_0_train.bin  


In [7]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

#### Custom PyTorch Dataset

In [8]:
""" Regression Dataset """
class DGLDatasetReg(torch.utils.data.Dataset):
    def __init__(self, address, transform=None, train=False, scaler=None):
            self.train = train
            self.scaler = scaler
            self.data_set, train_labels_masks_globals = dgl.load_graphs(address+".bin")
            num_graphs = len(self.data_set)
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
            self.transform = transform

    def scaler_method(self):
        if self.train:
            scaler = StandardScaler().fit(self.labels)
            self.scaler = scaler
        return self.scaler

    def __len__(self):
        return len(self.data_set)

    def __getitem__(self, idx):
        return  self.data_set[idx], torch.tensor(self.scaler.transform(self.labels)[idx]).float(), self.masks[idx], self.globals[idx]

#### Defining Train, Validation and Test sets

In [9]:
path_data_temp =  'FreeSolv_dataset/scaffold_0'
train_set = DGLDatasetReg(address=path_data_temp+"_train", train=True)
scaler = train_set.scaler_method()
val_set = DGLDatasetReg(address=path_data_temp+"_val", scaler=scaler)
test_set = DGLDatasetReg(address=path_data_temp+"_test", scaler=scaler)

print('Train set size: ', len(train_set))
print('Validation set size: ', len(val_set))
print('Test set size: ', len(test_set))

Train set size:  513
Validation set size:  64
Test set size:  65


#### DataLoader

In [10]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=True,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=True,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=True,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [11]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Some variables

In [12]:
#FreeSolv dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


#### Function in order to computing score(RMSE)

In [13]:
import math

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    loss_sum = nn.MSELoss(reduction='sum') # MSE with sum instead of mean, i.e., sum_i[(y_i)^2-(y'_i)^2]
    final_loss = 0
    with torch.no_grad():
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.tensor(scaler.inverse_transform(prediction.detach().cpu()))
            labels = torch.tensor(scaler.inverse_transform(labels.cpu()))
            loss = loss_sum(prediction, labels)
            final_loss += loss.item()
        final_loss /= val_size
        final_loss = math.sqrt(final_loss) # RMSE
    return final_loss / num_tasks


#### Loss Function

In [14]:
def loss_func(output, label, mask, num_tasks):
    criterion = nn.MSELoss(reduction='none')
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training

In [15]:
# Training function

def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations

    return epoch_train_loss


def train_evaluate(model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = np.Inf
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val < best_val:
                best_val = score_val
                print("Save checkpoint!")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [16]:
import time
start_time = time.time()

def test_evaluate(model):
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    test_score = compute_score(model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))

# GNN

#### GCN with two convolutional layers.

In [17]:
class GCN1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

##### Train the models and evaluate their performance

In [18]:
model_1 = GCN1(config, global_size, num_tasks)

In [19]:
train_evaluate(model_1)
test_evaluate(model_1)

Save checkpoint!
Epoch: 1/100 | Training Loss: 0.987 | Valid Score: 3.810
 
Epoch: 1/100 | Best Valid Score Until Now: 3.810 

Patience 1
Epoch: 2/100 | Training Loss: 0.965 | Valid Score: 3.836
 
Epoch: 2/100 | Best Valid Score Until Now: 3.810 

Patience 2
Epoch: 3/100 | Training Loss: 0.950 | Valid Score: 3.867
 
Epoch: 3/100 | Best Valid Score Until Now: 3.810 

Patience 3
Epoch: 4/100 | Training Loss: 0.929 | Valid Score: 3.889
 
Epoch: 4/100 | Best Valid Score Until Now: 3.810 

Patience 4
Epoch: 5/100 | Training Loss: 0.920 | Valid Score: 3.904
 
Epoch: 5/100 | Best Valid Score Until Now: 3.810 

Patience 5
Epoch: 6/100 | Training Loss: 0.909 | Valid Score: 3.907
 
Epoch: 6/100 | Best Valid Score Until Now: 3.810 

Patience 6
Epoch: 7/100 | Training Loss: 0.898 | Valid Score: 3.914
 
Epoch: 7/100 | Best Valid Score Until Now: 3.810 

Patience 7
Epoch: 8/100 | Training Loss: 0.888 | Valid Score: 3.917
 
Epoch: 8/100 | Best Valid Score Until Now: 3.810 

Patience 8
Epoch: 9/100 | 

##### GCN with 4 convolutional layers and batch normalization layers and dropouts.

In [20]:
class GCN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [21]:
model_2 = GCN2(config, global_size, num_tasks)

In [22]:
train_evaluate(model_2)
test_evaluate(model_2)

Save checkpoint!
Epoch: 1/100 | Training Loss: 1.405 | Valid Score: 4.262
 
Epoch: 1/100 | Best Valid Score Until Now: 4.262 

Save checkpoint!
Epoch: 2/100 | Training Loss: 1.319 | Valid Score: 4.135
 
Epoch: 2/100 | Best Valid Score Until Now: 4.135 

Save checkpoint!
Epoch: 3/100 | Training Loss: 1.198 | Valid Score: 4.133
 
Epoch: 3/100 | Best Valid Score Until Now: 4.133 

Patience 1
Epoch: 4/100 | Training Loss: 1.108 | Valid Score: 4.182
 
Epoch: 4/100 | Best Valid Score Until Now: 4.133 

Patience 2
Epoch: 5/100 | Training Loss: 1.104 | Valid Score: 4.214
 
Epoch: 5/100 | Best Valid Score Until Now: 4.133 

Patience 3
Epoch: 6/100 | Training Loss: 1.050 | Valid Score: 4.233
 
Epoch: 6/100 | Best Valid Score Until Now: 4.133 

Patience 4
Epoch: 7/100 | Training Loss: 1.026 | Valid Score: 4.170
 
Epoch: 7/100 | Best Valid Score Until Now: 4.133 

Patience 5
Epoch: 8/100 | Training Loss: 1.014 | Valid Score: 4.144
 
Epoch: 8/100 | Best Valid Score Until Now: 4.133 

Save checkpoin

#### GraphSAGE

In [23]:
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [24]:
class GraphSAGE1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [25]:
model_3 = GraphSAGE1(config, global_size, num_tasks)

In [26]:
train_evaluate(model_3)
test_evaluate(model_3)

Save checkpoint!
Epoch: 1/100 | Training Loss: 1.004 | Valid Score: 4.383
 
Epoch: 1/100 | Best Valid Score Until Now: 4.383 

Save checkpoint!
Epoch: 2/100 | Training Loss: 0.989 | Valid Score: 4.350
 
Epoch: 2/100 | Best Valid Score Until Now: 4.350 

Save checkpoint!
Epoch: 3/100 | Training Loss: 0.978 | Valid Score: 4.322
 
Epoch: 3/100 | Best Valid Score Until Now: 4.322 

Save checkpoint!
Epoch: 4/100 | Training Loss: 0.967 | Valid Score: 4.298
 
Epoch: 4/100 | Best Valid Score Until Now: 4.298 

Save checkpoint!
Epoch: 5/100 | Training Loss: 0.954 | Valid Score: 4.279
 
Epoch: 5/100 | Best Valid Score Until Now: 4.279 

Save checkpoint!
Epoch: 6/100 | Training Loss: 0.943 | Valid Score: 4.259
 
Epoch: 6/100 | Best Valid Score Until Now: 4.259 

Save checkpoint!
Epoch: 7/100 | Training Loss: 0.930 | Valid Score: 4.242
 
Epoch: 7/100 | Best Valid Score Until Now: 4.242 

Save checkpoint!
Epoch: 8/100 | Training Loss: 0.923 | Valid Score: 4.228
 
Epoch: 8/100 | Best Valid Score Unt

#### GraphSAGE with 4 layers and batch normalization and dropouts.

In [27]:
class GraphSAGE2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [28]:
model_4 = GraphSAGE2(config, global_size, num_tasks)

In [29]:
train_evaluate(model_4)
test_evaluate(model_4)

Save checkpoint!
Epoch: 1/100 | Training Loss: 1.042 | Valid Score: 4.343
 
Epoch: 1/100 | Best Valid Score Until Now: 4.343 

Save checkpoint!
Epoch: 2/100 | Training Loss: 0.878 | Valid Score: 4.308
 
Epoch: 2/100 | Best Valid Score Until Now: 4.308 

Save checkpoint!
Epoch: 3/100 | Training Loss: 0.793 | Valid Score: 4.261
 
Epoch: 3/100 | Best Valid Score Until Now: 4.261 

Save checkpoint!
Epoch: 4/100 | Training Loss: 0.695 | Valid Score: 4.190
 
Epoch: 4/100 | Best Valid Score Until Now: 4.190 

Save checkpoint!
Epoch: 5/100 | Training Loss: 0.651 | Valid Score: 4.123
 
Epoch: 5/100 | Best Valid Score Until Now: 4.123 

Save checkpoint!
Epoch: 6/100 | Training Loss: 0.595 | Valid Score: 4.051
 
Epoch: 6/100 | Best Valid Score Until Now: 4.051 

Save checkpoint!
Epoch: 7/100 | Training Loss: 0.571 | Valid Score: 3.983
 
Epoch: 7/100 | Best Valid Score Until Now: 3.983 

Save checkpoint!
Epoch: 8/100 | Training Loss: 0.523 | Valid Score: 3.923
 
Epoch: 8/100 | Best Valid Score Unt

#### Custom GNN

In [50]:
class CustomGraphConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(CustomGraphConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.u_add_v("h", "h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [51]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = CustomGraphConv(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = CustomGraphConv(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = CustomGraphConv(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = CustomGraphConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [52]:
model_5 = GNN(config, global_size, num_tasks)

In [53]:
train_evaluate(model_5)
test_evaluate(model_5)

Save checkpoint!
Epoch: 1/100 | Training Loss: 1.269 | Valid Score: 4.363
 
Epoch: 1/100 | Best Valid Score Until Now: 4.363 

Patience 1
Epoch: 2/100 | Training Loss: 1.000 | Valid Score: 4.446
 
Epoch: 2/100 | Best Valid Score Until Now: 4.363 

Patience 2
Epoch: 3/100 | Training Loss: 0.851 | Valid Score: 4.367
 
Epoch: 3/100 | Best Valid Score Until Now: 4.363 

Save checkpoint!
Epoch: 4/100 | Training Loss: 0.751 | Valid Score: 4.132
 
Epoch: 4/100 | Best Valid Score Until Now: 4.132 

Save checkpoint!
Epoch: 5/100 | Training Loss: 0.651 | Valid Score: 3.862
 
Epoch: 5/100 | Best Valid Score Until Now: 3.862 

Save checkpoint!
Epoch: 6/100 | Training Loss: 0.611 | Valid Score: 3.722
 
Epoch: 6/100 | Best Valid Score Until Now: 3.722 

Save checkpoint!
Epoch: 7/100 | Training Loss: 0.583 | Valid Score: 3.652
 
Epoch: 7/100 | Best Valid Score Until Now: 3.652 

Patience 1
Epoch: 8/100 | Training Loss: 0.524 | Valid Score: 3.688
 
Epoch: 8/100 | Best Valid Score Until Now: 3.652 

Pa